Задача обратного интерполирования

Разработчики: Денисов Лев и Дмитрий Калинин

Описание задачи, которую решает программа
Программа решает задачу обратного интерполирования. Исходные данные задаются таблично. Задача состоит в нахождении значения аргумента для заданного значения функции.

Используемые численные методы
1. Метод интерполяционного многочлена Лагранжа.
2. Метод секущих для решения алгебраических уравнений

In [2]:
import approximation_methods
import math

Функция get_p_l
Вычисляет значение интерполяционного многочлена Лагранжа в точке x.

Параметры:
x: Точка, в которой нужно вычислить значение многочлена.
key_values: Список ключевых значений (узлов интерполяции и соответствующих значений функции).

In [3]:
def get_p_l(x, key_values):
    return sum(math.prod([x - j[0] for j in key_values if j[0] != i]) /
               math.prod([i - j[0] for j in key_values if j[0] != i]) * value for i, value in key_values)

Функция f(x)
Определяет интерполируемую функцию.

In [4]:
def f(x):
    return 2 * math.sin(x) - x / 2

Функция creating_table(f, a, b, m1)
Создаёт таблицу значений функции для интерполяции.

Параметры:
f: Функция для табличного представления.
a: Начало интервала.
b: Конец интервала.
m1: Количество узлов интерполяции.

In [5]:
def creating_table(f, a, b, m1):
    h = (b - a) / (m1 - 1)
    d = {}
    for i in range(m1):
        x_i = a + h * i
        d[x_i] = f(x_i)

    print("{:<30} {:<30}".format("Узел", "Значение функции f"))
    for k, v in sorted(d.items(), key=lambda x: x[0]):
        print("{:<30} {:<30}".format(k, v))
    return d

Функция first_method(f, n, F, initial_table)
Реализует первый метод обратного интерполирования.

Параметры:
f: Исходная функция.
n: Степень интерполяционного многочлена.
F: Значение функции, для которого ищется аргумент.
initial_table: Исходная таблица значений функции.

In [6]:
def first_method(f, n, F, initial_table):
    modified_table = sorted([(i[1], i[0]) for i in initial_table.items()], key=lambda z: abs(F - z[0]))[:n + 1]
    print("\nОтраженная таблица")
    print("{:<30} {:<30}".format("Значение функции f", "Аргумент"))
    for k, v in modified_table:
        print("{:<30} {:<30}".format(k, v))
    x = get_p_l(F, modified_table)
    print("\nЗначение аргумента, найденного при помощи представления в форме Лагранжа: {}".format(x))
    r_n = abs(f(x) - F)
    print("Абсолютная величина невязки: " + str(r_n))

Функция secant_method(a, b, p_n, eps)
Метод секущих для решения уравнения p_n(x) = 0 на интервале [a, b] с точностью eps.

Параметры:
a: Начало интервала.
b: Конец интервала.
p_n: Функция, для которой ищется корень.
eps: Точность вычислений.

In [7]:
def secant_method(a, b, f, eps, *args):
    steps = 0
    prev, cur = a, b
    while abs(cur - prev) >= eps:
        if steps > 100:
            return -100, -1, -1
        steps += 1
        prev, cur = cur, cur - (f(cur) / (f(cur) - f(prev))) * (cur - prev)
    return steps, cur, abs(cur - prev), (a, b)

Функция second_method(f, a, b, n, F, eps, initial_table)
Реализует второй метод обратного интерполирования.

Параметры:
f: Исходная функция.
a: Начало интервала.
b: Конец интервала.
n: Степень интерполяционного многочлена.
F: Значение функции, для которого ищется аргумент.
eps: Точность вычислений.
initial_table: Исходная таблица значений функции.

In [8]:
def second_method(f, a, b, n, F, eps, initial_table):
    modified_table = sorted(initial_table.items(), key=lambda z: abs(F - z[1]))[:n + 1]
    print("\nОбновленная таблица")
    print("{:<30} {:<30}".format("Узел", "Значение функции f"))
    for k, v in modified_table:
        print("{:<30} {:<30}".format(k, v))
    p_n = lambda x: get_p_l(x, modified_table) - F
    steps, approximated_x, diff, first_approximation = secant_method(a, b, p_n, eps)
    print("Начальное приближение: " + str(first_approximation))
    if steps < 0:
        print("За " + str(abs(steps)) + " решение с заданной точностью не было найдено")
        return
    print("Количество шагов: " + str(steps))
    print("Приближенное решение: " + str(approximated_x))
    print("Расстояние до предыдущего приближения: " + str(abs(diff)))
    r_n = abs(f(approximated_x) - F)
    print("Абсолютная величина невязки: " + str(r_n))

In [10]:
print("Задача обратного интерполирования\nВариант 8")
print("Условия варианта 8: f(x)=2·sin(x) – x/2")  # a=0,2  b=0,7  m+1=11  n=8
m1 = int(input("Введите число значений в таблице (в наших обозначениях это m+1): "))
a, b = float(input("Введите левую границу отрезка [a, b]: ")), \
    float(input("Введите правую границу отрезка [a, b]: "))
print("\n")
initial_table = creating_table(f, a, b, m1)
com = ''
while True:
    if com == "exit":
        break
    n = int(input("Введите степень интерполяционного многочлена n ≤ {}: ".format(m1 - 1)))
    while n >= m1:
        print("Ошибка. Указано неверное значение n")
        n = int(input("Введите степень интерполяционного многочлена n ≤ {}: ".format(m1 - 1)))
    F = float(input("Введите значение функции для обратного интерполирования: "))
    epsilon = 10 ** ((-1) * int(input("Введите точность y, степень точности 10 ** (-y): ")))
    print("\n Метод первый:")
    first_method(f, n, F, initial_table)
    print("\n Метод второй:")
    second_method(f, a, b, n, F, epsilon, initial_table)
    com = input("Нажмите Enter или введите exit, чтобы завершить программу: ")
    print("\n")

Задача обратного интерполирования
Вариант 8
Условия варианта 8: f(x)=2·sin(x) – x/2
Введите число значений в таблице (в наших обозначениях это m+1): 6
Введите левую границу отрезка [a, b]: 0
Введите правую границу отрезка [a, b]: 2


Узел                           Значение функции f            
0.0                            0.0                           
0.4                            0.578836684617301             
0.8                            1.0347121817990454            
1.2000000000000002             1.2640781719344527            
1.6                            1.1991472060830102            
2.0                            0.8185948536513634            
Введите степень интерполяционного многочлена n ≤ 5: 4
Введите значение функции для обратного интерполирования: 0.85
Введите точность y, степень точности 10 ** (-y): 8

 Метод первый:

Отраженная таблица
Значение функции f             Аргумент                      
0.8185948536513634             2.0                           
1.034